In [1]:
# wir benötigen wieder ein paar python imports...
import os
import sys
from os import environ

sys.path.append(os.path.abspath(".."))



from client.connector_client import ConnectorClient
from client.sparql_builder import *
from messaging.datamodel import OPCUAReadPayload

client = ConnectorClient(bootstrap_servers=[environ.get("KAFKA_BROKER")])

# graphdb queries, um den richtigen connector zu identifizieren
await client.load_connector_config()
client.return_connectors()

dict_keys(['milling_station', 'assembly_station', 'injection_molding'])

In [2]:
# wechsel an den ausgewählten, richtigen connector
#await client.switch_connector('opcua_dev_connector')
await client.switch_connector("milling_station")

In [5]:
read_payload = OPCUAReadPayload(
    device_origin="client",
    nodeid_list = ["ns=2;i=59"],
)
await client.send_read_and_await_response(read_payload)    

type='OPCUAReadResponsePayload' device_origin='milling_station' value_dict={'ns=2;i=59': {'NamespaceUri': 'http://www.opcfoundation.org/UA/units/un/cefact', 'UnitId': 5408652, 'DisplayName': {'Encoding': 0, 'Locale': None, 'Text': 'millimetre per minute'}, 'Description': {'Encoding': 0, 'Locale': None, 'Text': 'millimetre per minute'}}}


{'status': 'success',
 'device_origin': 'milling_station',
 'message': '✅ Read successful from milling_station',
 'values': {'ns=2;i=59': {'NamespaceUri': 'http://www.opcfoundation.org/UA/units/un/cefact',
   'UnitId': 5408652,
   'DisplayName': {'Encoding': 0,
    'Locale': None,
    'Text': 'millimetre per minute'},
   'Description': {'Encoding': 0,
    'Locale': None,
    'Text': 'millimetre per minute'}}}}

In [5]:
await client.list_instances(class_uri="df:DataPoint")

,instance
0,e179af64-421c-4c0e-af05-221471a35190
1,a8f37154-69bb-477a-8ea9-6d10f726d4a1
2,e65844af-31af-44b4-9b1f-c72e01d3a608
3,85f8b92c-d14f-4806-8dce-eb44627cee40
4,cf43a814-76c2-4286-aa18-f96a32a8935f
...,...
145,7bf3a337-0503-48d1-83e4-1b5c6b6c8e9f
146,73be8dfa-030c-4f57-a855-7f70668ff642
147,a135c9fc-c928-4719-a9c5-67b48dea3aaa
148,1c865e9b-d7d9-4218-a9a3-2cf55460cb8a


In [9]:
datapoint_uri = "http://stephantrattnig.org/instances#c6a5097d-79e4-4edc-b199-a0e8a6bac38e"
await client.resolve_and_read_datapoint(datapoint_uri=datapoint_uri)

type='OPCUAReadResponsePayload' device_origin='opcua_dev_connector' value_dict={'ns=2;i=59': 59.929101890120016}


{'status': 'success',
 'device_origin': 'opcua_dev_connector',
 'message': '✅ Read successful from opcua_dev_connector',
 'values': {'ns=2;i=59': 59.929101890120016}}

# deprecated sparql stuff

In [ ]:
df_instance = "http://stephantrattnig.org/instances#"

In [ ]:
out = df_instance + "f5b88211-0c7a-4ce6-8e98-5613ff846686"

In [ ]:
query = queries.build_get_properties_query(subject_uri=out)
await client.query_graphdb(query)

In [ ]:
queries.get_all_classes()
await client.query_graphdb(queries.get_all_classes())

In [ ]:
property_uri = "rdfs:label"
value = "DataPoint: CycleTime"
query = queries.build_search_by_property_query(property_uri=property_uri, value=value)
await client.query_graphdb(query)

In [ ]:
query = queries.build_search_entity_query("DataPoint")
await client.query_graphdb(query)

In [ ]:
query = queries.search_entity_query(keyword="", class_uri = None)
await client.query_graphdb(query)

In [ ]:
query = queries.get_all_classes()
await client.query_graphdb(query)

In [ ]:
subject_uri = "http://stephantrattnig.org/instances#bd79be83-7900-4872-8019-cf1f03fed5a0"
query = queries.build_get_properties_query(subject_uri = subject_uri)
await client.query_graphdb(query)

In [ ]:
print(query)

In [ ]:
device_uri = "http://stephantrattnig.org/instances#d7006e5d-136d-4d3e-af73-ffd4cbaf2aa1"
device_props = [
    "rdfs:label",
    "df:deviceIdentifier",
    "df:deviceManufacturer",
    "df:deviceModelName",
    "df:deviceAddress"
]

query = queries.build_get_properties_query(subject_uri=device_uri, property_uris=device_props)
await client.query_graphdb(query)

In [ ]:
print(query)

In [3]:
query_builder = Queries()
query = query_builder.build_list_instances_query(
    class_uri="df:Subscription",
    optional_props=[
        "df:subscriptionStart",
        "df:subscriptionEnd",
        "df:hasSubscriptionState"
    ]
)
print(query)

PREFIX df: <http://stephantrattnig.org/data_fabric_ontology#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX df_instance: <http://stephantrattnig.org/instances#>
        

        SELECT ?instance ?subscriptionStart ?subscriptionEnd ?hasSubscriptionState
        WHERE {
            ?instance a df:Subscription .
            OPTIONAL { ?instance df:subscriptionStart ?subscriptionStart . }
OPTIONAL { ?instance df:subscriptionEnd ?subscriptionEnd . }
OPTIONAL { ?instance df:hasSubscriptionState ?hasSubscriptionState . }

        }


In [4]:
query = queries.build_get_related_query(
    subject_uri="http://stephantrattnig.org/instances#d7006e5d-136d-4d3e-af73-ffd4cbaf2aa1",
    predicate_uri="http://stephantrattnig.org/data_fabric_ontology#hasDataPoint"
)
await client.query_graphdb(query)

""


In [5]:
query = queries.build_get_related_inverse_query(
    object_uri="http://stephantrattnig.org/instances#73d6913a-9c31-4c19-9b60-e0fb844666dc",
    predicate_uri="http://stephantrattnig.org/data_fabric_ontology#subscribesToDataPoint",
    optional_props=[
        "http://stephantrattnig.org/data_fabric_ontology#subscriptionStart",
        "http://stephantrattnig.org/data_fabric_ontology#subscriptionEnd",
        "http://stephantrattnig.org/data_fabric_ontology#hasSubscriptionState"
    ]
)
await client.query_graphdb(query)

""


In [6]:
print(query)

PREFIX df: <http://stephantrattnig.org/data_fabric_ontology#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        
        SELECT ?subject ?label ?subscriptionStart ?subscriptionEnd ?hasSubscriptionState WHERE {
            ?subject <http://stephantrattnig.org/data_fabric_ontology#subscribesToDataPoint> <http://stephantrattnig.org/instances#73d6913a-9c31-4c19-9b60-e0fb844666dc> .
            OPTIONAL { ?subject rdfs:label ?label }
OPTIONAL { ?subject <http://stephantrattnig.org/data_fabric_ontology#subscriptionStart> ?subscriptionStart }
OPTIONAL { ?subject <http://stephantrattnig.org/data_fabric_ontology#subscriptionEnd> ?subscriptionEnd }
OPTIONAL { ?subject <http://stephantrattnig.org/data_fabric_ontology#hasSubscriptionState> ?hasSubscriptionState }

        }


In [7]:
query = queries.build_list_instances_query(
    class_uri="df:Topic",
    optional_props=["df:topicName", "df:hasTopicType"]
)
await client.query_graphdb(query)

,instance,topicName,hasTopicType
0,request.ServiceModuleOPCUA.opcua_dev_connector,request.ServiceModuleOPCUA.opcua_dev_connector,requestType
1,response.ServiceModuleOPCUA.opcua_dev_connector,response.ServiceModuleOPCUA.opcua_dev_connector,responseType
2,telemetry.ServiceModuleOPCUA.opcua_dev_connector,telemetry.ServiceModuleOPCUA.opcua_dev_connector,telemetryType


In [8]:
query = queries.build_list_instances_query(
    class_uri="df:Connector",
    optional_props=[
        "rdfs:label",
        "df:ConnectorAddress",
        "df:moduleId",
        "df:moduleType"
    ]
)
await client.query_graphdb(query)

,instance,label,ConnectorAddress,moduleId,moduleType
0,d5e80e76-1048-4c42-ac1b-188abb3d50ff,Connector: opcua_dev_connector,10.157.101.85,opcua_dev_connector,ServiceModuleOPCUA
1,748302f0-7559-47f9-a799-0930c681b259,Connector: opcua_dev_connector,10.157.101.85,opcua_dev_connector,ServiceModuleOPCUA


In [19]:
query = queries.build_list_instances_query(class_uri="df:DataPoint", optional_props=["df:hasDataType", "rdfs:label"])
await client.query_graphdb(query)

,instance,hasDataType,label
0,35af9f26-4818-458b-9a44-2f15cd140b88,float,DataPoint: AlignmentOffset
1,12281921-0815-4373-ba4e-1743bf4d4f77,float,DataPoint: ClampingForce
2,f87e73ae-f4c4-4ec4-91df-23776a6e40c2,float,DataPoint: ScrewdriverTorque
3,33573786-e874-4508-b0b3-e2a81741d92d,float,DataPoint: CycleTime
4,8a335dee-5c7c-4d63-9391-35ae253ab998,float,DataPoint: MaxClampingForce
5,a346b5e3-e7a8-428e-b663-c42be2630ff9,float,DataPoint: TorqueThreshold
6,026d047a-e6de-4ef8-ad7e-8ce915b048a5,float,DataPoint: AlignmentOffset
7,1cd10425-c663-4b5a-ac36-5632d35c45c6,float,DataPoint: ClampingForce
8,b1c708d6-a010-4099-aa17-a44a92379b16,float,DataPoint: ScrewdriverTorque
9,0cb14aed-4ad3-4ac6-9ff1-e2026eda590f,float,DataPoint: CycleTime


In [20]:
query = queries.build_list_instances_query("df:DataPoint", optional_props=["rdfs:label"])
output = await client.query_graphdb(query)
print(output)

                                instance                         label
0   35af9f26-4818-458b-9a44-2f15cd140b88    DataPoint: AlignmentOffset
1   12281921-0815-4373-ba4e-1743bf4d4f77      DataPoint: ClampingForce
2   f87e73ae-f4c4-4ec4-91df-23776a6e40c2  DataPoint: ScrewdriverTorque
3   33573786-e874-4508-b0b3-e2a81741d92d          DataPoint: CycleTime
4   8a335dee-5c7c-4d63-9391-35ae253ab998   DataPoint: MaxClampingForce
5   a346b5e3-e7a8-428e-b663-c42be2630ff9    DataPoint: TorqueThreshold
6   026d047a-e6de-4ef8-ad7e-8ce915b048a5    DataPoint: AlignmentOffset
7   1cd10425-c663-4b5a-ac36-5632d35c45c6      DataPoint: ClampingForce
8   b1c708d6-a010-4099-aa17-a44a92379b16  DataPoint: ScrewdriverTorque
9   0cb14aed-4ad3-4ac6-9ff1-e2026eda590f          DataPoint: CycleTime
10  3cb906ed-0483-4abd-836d-39ba81f2717a   DataPoint: MaxClampingForce
11  3c3cc7f7-f136-41d7-a802-3403a309be5d    DataPoint: TorqueThreshold


In [21]:
selected_datapoint = output.iloc[1]["instance"]
selected_datapoint_uri = f"http://stephantrattnig.org/instances#{selected_datapoint}"
query = queries.build_get_properties_query(selected_datapoint_uri)
output = await client.query_graphdb(query)
print(output)

                      p                            o
0                  type                    DataPoint
1                 label     DataPoint: ClampingForce
2      allowsCapability               ReadCapability
3      allowsCapability          SubscribeCapability
4   dataPointIdentifier                    ns=2;i=59
5         dataPointName                ClampingForce
6         hasAccessMode               ReadCapability
7         hasAccessMode          SubscribeCapability
8           hasDataType                        float
9           hasMetadata  OPC UA Type: AnalogItemType
10              hasUnit                       newton


In [17]:
print(selected_datapoint)

12281921-0815-4373-ba4e-1743bf4d4f77
